In [1]:
from pyfasta import Fasta

In [2]:
import os
from pyfasta import Fasta
from time import strftime
import re

In [3]:
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function
from builtins import range,object

In [4]:
import sys
if sys.version_info.major ==2:
	import cPickle as pickle
else:
	import pickle
	from sys import intern
import os
from pyfasta import Fasta
from time import strftime
import re


In [5]:
__author__ = 'Zhengtao Xiao'
__version__ = "1.2.15"

In [14]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

def percentage_format(value,decimal=2):
	"""
	Format number as percentage
	"""
	formatStr = "{:." + str(decimal) + "%}"
	return formatStr.format(value)
	#return "{:.2%}".format(value)

def get_colors(NUM_COLORS):
	cm = plt.get_cmap('gist_rainbow')
	colors = []
	for i in range(NUM_COLORS):
		colors.append(cm(1.*i/NUM_COLORS))  # color will now be an RGBA tuple
	return colors

def pie(orderDict, outname):
	sizes = list(orderDict.values())
	total = sum(sizes)
	labels = []
	for i in orderDict.keys():
		perct = orderDict[i] / float(total)
		labels.append("%s: %i (%s)" % (i,int(orderDict[i]),percentage_format(perct,1)))

	colors = get_colors(len(labels))
	plt.figure(figsize=(8,6))
	patches, text = plt.pie(sizes, startangle=90,colors=colors)
	plt.legend(patches,labels,loc=4,bbox_to_anchor=(1, 0),fontsize=15)
	plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
	plt.tight_layout()
	plt.savefig(outname + "_ORFs_category.pdf")
	plt.close()

In [15]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import h5py
import os

def generate_colors(length,shiftn=0):
	"""
	generate r,b,g colors
	"""
	colors0 = ["red","blue","green"]
	colors1 = ["green","red","blue"]
	colors2 = ["blue","green","red"]
	rep_times = int(np.ceil(length/3))
	if shiftn == 0:
		colors = np.tile(colors0,rep_times)
	elif shiftn==1:
		colors = np.tile(colors1,rep_times)
	else:
		colors = np.tile(colors2,rep_times)
	return colors[:length]

def plot_ORF(ax,tpsites,orf_start,orf_stop,transcript_id,StartCodon):
	colors = generate_colors(tpsites.size,orf_start%3)
	ax.vlines(np.arange(tpsites.size),ymin=0,ymax=tpsites,colors=colors,linewidth=0.3)
	ax.tick_params(axis='x',which="both",top=False,direction='out',labelsize=15)
	ax.tick_params(axis='y',which="both",labelsize=15)
	ax.set_ylabel("P-site reads density",fontsize=18)
	ax.set_xlim((0,tpsites.size))
	ax.set_title(transcript_id+":"+str(orf_start)+"-"+str(orf_stop)+":"+StartCodon)

def plot_predicted(tpsites,orf_start,orf_stop,transcript_id,StartCodon,outname):
	fig=plt.figure(figsize=(8,4))
	## ax1
	ax1=fig.add_subplot(111)
	tpsites_orf=tpsites[int(orf_start)-1:orf_stop]
	colors = generate_colors(tpsites.size,orf_start%3)
	ax1.vlines(np.arange(tpsites_orf.size),ymin=0,ymax=tpsites_orf,colors=colors[int(orf_start)-1:orf_stop],linewidth=0.3)
	ax1.tick_params(axis='x',which="both",top=False,direction='out',labelsize=15)
	ax1.tick_params(axis='y',which="both",labelsize=15)
	ax1.set_ylabel("P-site reads density",fontsize=18)
	ax1.set_xlim((0,tpsites_orf.size))
	ax1.set_title(transcript_id+":"+str(orf_start)+"-"+str(orf_stop)+":"+StartCodon)
	plt.savefig(outname + ".pdf")


def plot_annotation(ax,tlength,start,stop,label,color):
	width = 0.15
	ax.set_xlim((0,tlength))
	ax.fill((start-1,stop,stop,start-1),
			(1+width/2,1+width/2,1-width/2,1-width/2),
			color=color,lw=0.5,zorder=20)
	ax.axhline(1,color="gray",lw=0.5)
	ax.set_frame_on(False)
	ax.xaxis.set_ticks_position("none")
	ax.yaxis.set_ticks_position("none")
	ax.set_xticks([])
	ax.set_yticks([1])
	ax.set_yticklabels([label],fontsize=18)
	ax.set_ylim((1-width/2,1+width/2))

def read_psites_array(filename,transcript_id):
	with h5py.File(filename,"r") as fin:
		k=np.array([i.decode("utf-8") if type(i)==bytes else i for i in fin["transcript_ids"][:]])
		idx = np.where(k == transcript_id)[0][0]
		v = fin["p_sites"][idx]
	return v

def plot_main(cds_start,cds_end,psites_array,orf_tstart,orf_tstop,transcript_id,StartCodon,outname):
	"""
	the main plot function
	"""
	plt.figure(figsize=(8,4))
	if cds_start is not None:
		gs = gridspec.GridSpec(3,1,height_ratios=[10,1,1],hspace=0.6,left=0.2,right=0.95)
	else:
		gs = gridspec.GridSpec(2,1,height_ratios=[11,1],hspace=0.6,left=0.2,right=0.95)

	ax1 = plt.subplot(gs[0])
	ax2 = plt.subplot(gs[1])
	plot_ORF(ax1,psites_array,orf_tstart,orf_tstop,transcript_id,StartCodon)
	plot_annotation(ax2,psites_array.size,orf_tstart,orf_tstop,"Predicted","#3994FF")

	if cds_start is not None:
		ax3 = plt.subplot(gs[2])
		plot_annotation(ax3,psites_array.size,cds_start,cds_end,"Annotated","#006DD5")
	# plt.tight_layout()
	plt.savefig(outname + ".pdf")

def main():
	from .parsing_opts import parsing_plot_orf_density
	from .loadconfig import LoadConfig
	args = parsing_plot_orf_density()
	transcripts_cds_file = os.path.join(args.annot_dir,"transcripts_cds.txt")
	cds_start = None
	cds_end = None
	with open(transcripts_cds_file) as fin:
		for line in fin:
			if line.startswith(args.transcript_id):
				_,s,e = line.strip().split("\t")
				cds_start = int(s)
				cds_end = int(e)
				break
	configIn = LoadConfig(args.config_file)
	samples = [i["samplename"] for i in configIn.configList]
	for i,s in enumerate(samples):
		if i == 0:
			psites_array = read_psites_array(s + "_psites.hd5",args.transcript_id)
		else:
			psites_array += read_psites_array(s + "_psites.hd5",args.transcript_id)
	if not args.outname:
		args.outname = "%s_%i_%i" % (args.transcript_id,args.orf_tstart,args.orf_tstop)
	if args.PlotAnnotatedORF.upper()=="YES":
		plot_main(cds_start,cds_end,psites_array,args.orf_tstart,args.orf_tstop,args.transcript_id,args.StartCodon,args.outname)
	elif args.PlotAnnotatedORF.upper() == 'NO':
		plot_predicted(psites_array,args.orf_tstart,args.orf_tstop,args.transcript_id,args.StartCodon,args.outname)
	else:
		raise IOError("Please reset your --plot-annotated-orf parameter: yes or no. default=yes")

In [6]:
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function
# -*- coding:UTF-8 -*-
__author__ = 'Zhengtao Xiao'

import argparse
import os

def read_file(f):
	if not os.path.exists(f):
		raise  ValueError("Error, file %s not found!\n" % f)
	fList = []
	with open(f) as fin:
		for line in fin:
			if line.startswith("#") or line.strip() == "":
				continue
			fList.append(line.strip())
	return fList

def parsing_gtf_update():
	parser = argparse.ArgumentParser(
		description="This script is designed for preparing the appropriate GTF file from \
					 custom GTF file (or those not from ENSEMBL/GENCODE database)"
	)
	# parser.add_argument("-g","--add",dest="",required=True,type=str,
	#					 help="If add the gene information")
	# parser.add_argument("-t","",dest="",
	#					 help="if add the transcript information")
	parser.add_argument("gtfFile")
	args = parser.parse_args()
	return args

def parsing_transcript():
	parser = argparse.ArgumentParser(
		description="This script is designed for preparing transcripts annotation files."
	)
	parser.add_argument("-g","--gtf",dest="gtfFile",required=True,type=str,
						help='Default, suitable for GENCODE and ENSEMBL GTF file, \
							  please refer: https://en.wikipedia.org/wiki/GENCODE, \
							  or using GTFupdate command to update your GTF file.')
	parser.add_argument("-f","--fasta",dest="genomeFasta",required=True,type=str,
						help="The genome sequences file in fasta format.")
	parser.add_argument("-o","--out_dir",required=True,type=str,dest="out_dir",help="annotation directory name.")
	parser.add_argument('-V',"--version",action="version",version=__version__)
	args = parser.parse_args()

	if not os.path.exists(args.out_dir):
		try:
			os.mkdir(args.out_dir)
		except OSError as e:
			raise e

	if not os.path.exists(args.gtfFile):
		raise ValueError("Error, gtf file not found:%s.\n" % args.gtfFile)

	if not os.path.exists(args.genomeFasta):
		raise ValueError("Error, genomic fasta not found: %s\n" % args.genomeFata)

	return args
def parsing_metaplots():
	parser = argparse.ArgumentParser(
		description="""
		This script create aggregate plots of distances from the 5'end of reads to start or stop codons,
		which help determine the length range of the PRF reads that are most likely originated from the
		translating ribosomes and identify the P-site locations for each reads lengths.
		"""
	)
	parser.add_argument("-a","--annot_dir",dest="annot_dir",required=True,type=str,
						help="transcripts annotation directory, generated by prepare_transcripts.")
	group = parser.add_mutually_exclusive_group(required=True)
	group.add_argument("-r","--rpf_mapping_file",dest="rpf_mapping_file",required=False,type=str,
						help="ribo-seq BAM/SAM file aligned to the transcriptome.")
	group.add_argument("-i","--input_file",dest="rpf_mapping_file",required=False,type=read_file,
						help="the file list the ribo-seq BAM/SAM files aligned to the transcriptome.")
	parser.add_argument("-s","--stranded",dest="stranded",required=False,type=str,choices=["yes","reverse","no"],
						default="yes",help="whether the data is strand-specific, \
						reverse means reversed strand interpretation.(default: yes)")
	parser.add_argument("-m","--minimum-length",dest="minLength",required=False,type=int,default=24,
						help="minimum length of read to output, default 24")
	parser.add_argument("-M","--maximum-length",dest="maxLength",required=False,type=int,default=36,
						help="maximum length of read to output, default 36")
	parser.add_argument("-pv1","--pvalue1_cutoff",dest="pvalue1_cutoff",required=False,type=float,default=0.001,
						help="pvalue cutoff of frame0 > frame2 for automatically predicting P-site location, default 0.001")
	parser.add_argument("-pv2","--pvalue2_cutoff",dest="pvalue2_cutoff",required=False,type=float,default=0.001,
						help="pvalue cutoff of frame0 > frame2 for automatically predicting P-site location, default 0.001")
	parser.add_argument("-f0_percent","--frame0_percent",dest="frame0_percent",required=False,type=float,default=0.65,
						help="proportion threshold of the number of reads in frame0, defined by f0/(f0+f1+f2), default 0.65")
	parser.add_argument("-o","--outname",dest="outname",required=False,type=str,default="metaplots",
						help="name of output pdf file(default: metaplots)")
	parser.add_argument('-V',"--version",action="version",version=__version__)
	args = parser.parse_args()

	if not os.path.exists(args.annot_dir):
		raise ValueError("Error, the transcript annotation directory not found: {} \n \
						  pleas run prepare_transcripts.py first.".format(args.annot_dir))
	if args.minLength > args.maxLength:
		raise ValueError("minimum length must be <= maximum length (currently %d and %d, respectively)" % (args.minLength, args.maxLength))
	if args.minLength <= 0 or  args.maxLength <=0:
		raise ValueError("minimum length or maximum length must be larger than 0.")
	if type(args.rpf_mapping_file) is str:
		if not os.path.exists(args.rpf_mapping_file):
			raise  ValueError("Error, the rpf mapping file not found: %s\n" % args.rpf_mapping_file)
		args.rpf_mapping_file = [args.rpf_mapping_file]

	if args.stranded == "yes":
		args.stranded = True
	elif args.stranded == "reverse":
		args.stranded = False
	else:
		args.stranded = None
	args.pvalue1_cutoff = float(args.pvalue1_cutoff)
	args.pvalue2_cutoff = float(args.pvalue2_cutoff)
	args.frame0_percent = float(args.frame0_percent)

	return args

def parsing_ribo():
	parser = argparse.ArgumentParser(
		description="The main function designed for detecting ORF using ribosome-profiling data."
	)
	parser.add_argument("-a","--annot_dir",dest="annot_dir",required=True,type=str,
						help="transcripts annotation directory, generated by prepare_transcripts.")
	parser.add_argument("-c","--config_file",dest="config_file",required=True,type=str,
						help="list bam file and P-sites information in this file, \
						please refer to the example file in data folder.")
	# parser.add_argument("-n","--num-threads",dest="threads_num",default=1,required=False,
	#					 help="number of threads, optimal number is number of bam files.", type=int)
	parser.add_argument("-l","--longest-orf",dest="longest_orf",choices=["yes","no"],default="yes",required=False,
						help="Default: yes, the region from most distal AUG to stop was defined as an ORF. \
							  If set to no , the position of start codon will be automatically determined by program.", type=str)
	parser.add_argument("-s","--start_codon",default="ATG",type=str,dest="start_codon",
						help="The canonical start codon. default: ATG")
	parser.add_argument("-A","--alt_start_codons",default="",type=str,dest="alternative_start_codons",
						help="The alternative start codon, such as CTG,GTG, default: None. Multiple codons should be separated by comma.")
	parser.add_argument("-S","--stop_codon",default="TAA,TAG,TGA",type=str,dest="stop_codon",
						help="Stop codon, default: TAA,TAG,TGA")
	parser.add_argument("-t","--transl_table",default=1,dest="transl_table",type=int,
						help="ORF translation table(Default: 1). Assign the correct genetic code based on your organism, \
						[please refer: https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi]")
	parser.add_argument("-m","--min-AA-length",dest="min_AA_length",default="5",required=False,
						help="The minimal length of predicted peptides,default 5", type=int)
	parser.add_argument("--dependence_test", dest="dependence_test", required=False, type=str, default="none",
						choices=["none", "mic", "pcc"],
						help="the method for measuring the dependence between frame1 and frame2. \n \
							This test could help determine whether the combined p-values should be ajusted to account for the dependence between two test (i.e. F0 vs F1 and F0 vs F2). \n \
							mic: Maximal Information Coefficient; pcc: Pearson Correlation Coefficient.")
	parser.add_argument("--stouffer_adj", dest="stouffer_adj", required=False, type=str, default="none",
						choices=["none","nyholt", "liji", "gao", "galwey"],
						help="the method for adjustment the cominbed p-values to account for the dependence between two tests (i.e. F0 vs F1 and F0 vs F2). \n \
							see details at: https://search.r-project.org/CRAN/refmans/poolr/html/stouffer.html")
	parser.add_argument("-p","--pval-cutoff",dest="pval_cutoff",default=0.05,required=False,
						help="P-value cutoff for ORF filtering, default 0.05", type=float)
	parser.add_argument("--pval_adj", dest="pval_adj", required=False, type=str, default="fdr_bh",
						choices=["fdr_bh","bonferroni","sidak","holm-sidak","holm","simes-hochberg","hommel","fdr_by","fdr_tsbh","fdr_tsbky"],
						help="the method used to correct p-values for multiple testing. default: Benjamini/Hochberg method. \n \
							see details at: https://www.statsmodels.org/devel/generated/statsmodels.stats.multitest.multipletests.html#statsmodels.stats.multitest.multipletests")
	# parser.add_argument("-P","--parallel_num",dest="parallel_num",default=1,required=False,
	#					 help="the number of threads to read the alignment file(s), \
	#					 the optimal value is the number of alignment files, default=1",type=int)
	parser.add_argument("-g","--output-gtf",dest="output_gtf",action='store_true',default=False,required=False,
						help="output the gtf file of predicted ORFs")
	parser.add_argument("-b","--output-bed",dest="output_bed",action='store_true',default=False,required=False,
						help="output the bed file of predicted ORFs")
	parser.add_argument("-o","--output-name",dest="output_name",default="final_result",required=False,
						help="output file name, default: final_result", type=str)
	parser.add_argument('-V',"--version",action="version",version=__version__)
	args = parser.parse_args()

	if not os.path.exists(args.annot_dir):
		raise ValueError("Error, the transcript annotation directory not found: {} \n \
					 pls run prepare_transcripts.py first. ".format(args.annot_dir))

	return args

def parsing_plot_orf_density():
	parser = argparse.ArgumentParser(
		description="This script is designed for plot the P-site profile of specified ORF."
	)
	parser.add_argument("-a","--annot_dir",dest="annot_dir",required=True,type=str,
						help="transcripts annotation directory, generated by prepare_transcripts.")
	parser.add_argument("-c","--config_file",dest="config_file",required=True,
						help="defile bam file information in this file, \
						please refer to the example file in data folder.",type=str)
	parser.add_argument("-t","--transcript_id",dest="transcript_id",required=True,type=str,
						help="the transcript id")
	parser.add_argument("-s","--orf_tstart",dest="orf_tstart",required=True,type=int,
						help="transcript-level coordinates of start of ORF (orf_tstart)")
	parser.add_argument("-e","--orf_tstop",dest="orf_tstop",required=True,type=int,
						help="transcript-level coordinates of end of ORF (orf_tstop)")
	parser.add_argument("-o","--outname",dest="outname",required=False,type=str,default="",
						help="output file name,default is transcriptid_tstart_tstop.pdf")
	parser.add_argument("--start-codon",dest="StartCodon",required=True,type=str,
						help="start codon of predicted ORFs")
	parser.add_argument("--plot-annotated-orf",dest="PlotAnnotatedORF",required=False,type=str,default="yes",
						help="plot the annotated orf (yes) or not (no).")
	parser.add_argument('-V',"--version",action="version",version=__version__)
	args = parser.parse_args()

	args.orf_tstart = args.orf_tstart -1 # change to 0 based
	if not os.path.exists(args.annot_dir):
		raise ValueError("Error, the transcript annotation directory not found: {} \n \
					 pls run prepare_transcripts.py first. ".format(args.annot_dir))
	return args

def parsing_ORF_count():
	parser = argparse.ArgumentParser(
		description="This script is designed for calculating the number of reads mapping to ORF with the alignment files \
		in SAM/BAM format (aligned to genome) and a feature file in GTF format"
	)
	parser.add_argument("-s","--stranded",dest="stranded",required=False,type=str,choices=["yes","reverse","no"],
						default="yes",help="whether the data is strand-specific, \
						reverse means reversed strand interpretation. (default: yes)")
	parser.add_argument("-a","--minaqual",dest="min_quality",required=False,type=int,
						default=10,help="skip all reads with alignment quality lower than the given minimum value (default:10)")
	parser.add_argument("-c","--count_mode",dest="count_mode",required=False,type=str,choices=["union","intersection-strict"],
						default="intersection-strict",help="mode to handle reads overlapping more than one ORF (choices:\
						union,intersection-strict;default: intersection-strict)")
	parser.add_argument("-g","--gtf",dest="gtf_file",required=False,type=str,default="final_result_collapsed.gtf",
						help="ORF gtf file generated by RiboCode, default:final_result")
	parser.add_argument("-r","--rpf_mapping_file",dest="rpf_mapping_file",required=True,type=str,
						help="ribo-seq BAM/SAM file aligned to the genome, multiple files should be separated with \",\"")
	parser.add_argument("-f","--first_exclude_codons",dest="first_exclude_codons",required=False,type=int,default=15,
						help="excluding the reads aligned to the first few codons of the ORF, default:15")
	parser.add_argument("-l","--last_exclude_codons",dest="last_exclude_codons",required=False,type=int,default=5,
						help="excluding the reads aligned to the last few codons of the ORF, default:5")
	parser.add_argument("-e","--exclude_min_ORF",dest="exclude_min_ORF",required=False,type=int,default=100,
						help="the minimal length(nt) of ORF for excluding the reads aligned to first and last few codons, default:100")
	parser.add_argument("-m","--min_read",dest="min_read",required=False,type=int,default=26,
						help="minimal read length for the counting of RPF,default:26")
	parser.add_argument("-M","--max_read",dest="max_read",required=False,type=int,default=34,
						help="maximal read length for the counting of RPF,default:34")
	# parser.add_argument("-p","--parallel_num",dest="parallel_num",required=False,type=int,default=1,
	#					 help="the number of threads to read the alignment file(s), \
	#					 the optimal value is the number of alignment files, default=1")
	parser.add_argument("-o","--output",dest="output_file",required=False,type=str,
						default="-",help="write out all ORF counts into a file, default is to write to standard output")
	parser.add_argument('-V',"--version",action="version",version=__version__)
	args = parser.parse_args()

	if not os.path.exists(args.gtf_file):
		raise ValueError("Error, the gtf file not found: {}".format(args.gtf_file))
	if args.first_exclude_codons * 3 + args.last_exclude_codons * 3 >= args.exclude_min_ORF:
		raise ValueError("Error, the exclude_min_ORF is too small: %i" % args.exclude_min_ORF)
	if "," in args.rpf_mapping_file:
		rpf_mapping_files = [i.strip() for i in args.rpf_mapping_file.split(",")]
		args.rpf_mapping_file = rpf_mapping_files
	else:
		args.rpf_mapping_file = [args.rpf_mapping_file]
	# if args.parallel_num > len(args.rpf_mapping_file):
	# 	args.parallel_num = len(args.rpf_mapping_file)
	return args

def parsing_ribo_onestep():
	parser = argparse.ArgumentParser(
		description="The function contains the steps to prepare a reference genome, to determinate the P-site locations" +
					" and to detect ORFs. It automatically creates some of the output files."

	)
	parser.add_argument("-g","--gtf",dest="gtfFile",required=True,type=str,
						help='Default, suitable for GENCODE and ENSEMBL GTF file, \
							  please refer: https://en.wikipedia.org/wiki/GENCODE')
	parser.add_argument("-f","--fasta",dest="genomeFasta",required=True,type=str,
						help="The genome sequences file in fasta format.")
	group = parser.add_mutually_exclusive_group(required=True)
	group.add_argument("-r","--rpf_mapping_file",dest="rpf_mapping_file",required=False,type=str,
						help="ribo-seq BAM/SAM file aligned to the transcriptome.")
	group.add_argument("-i","--input_file",dest="rpf_mapping_file",required=False,type=read_file,
						help="the file list the ribo-seq BAM/SAM files aligned to the transcriptome.")
	parser.add_argument("-stranded","--stranded",dest="stranded",required=False,type=str,choices=["yes","reverse","no"],
						default="yes",help="whether the data is strand-specific, \
						reverse means reversed strand interpretation.(default: yes)")
	parser.add_argument("-m","--minimum-length",dest="minLength",required=False,type=int,default=24,
						help="minimum read length for metaplots analysis, default 24")
	parser.add_argument("-M","--maximum-length",dest="maxLength",required=False,type=int,default=36,
						help="maximum read length for metaplots analysis, default 36")
	parser.add_argument("-f0_percent","--frame0_percent",dest="frame0_percent",required=False,type=float,default=0.65,
						help="proportion threshold of reads in frame0. For automatically predicting P-site location, default 0.65")
	parser.add_argument("-l","--longest-orf",dest="longest_orf",choices=["yes","no"],default="yes",required=False,
						help="Default: yes, the region from most distal AUG to stop was defined as an ORF. \
							  If set to no , the position of start codon will be automatically determined by program.", type=str)
	parser.add_argument("-p","--pval-cutoff",dest="pval_cutoff",default=0.05,required=False,
						help="P-value cutoff for ORF filtering, default 0.05", type=float)
	parser.add_argument("-s","--start_codon",default="ATG",type=str,dest="start_codon",
						help="The canonical start codon. default: ATG")
	parser.add_argument("-A","--alt_start_codons",default="",type=str,dest="alternative_start_codons",
						help="The alternative start codon, such as CTG,GTG, default: None. Multiple codons should be separated by comma.")
	parser.add_argument("-S","--stop_codon",default="TAA,TAG,TGA",type=str,dest="stop_codon",
						help="Stop codon, default: TAA,TAG,TGA")
	parser.add_argument("-t","--transl_table",default=1,dest="transl_table",type=int,
						help="ORF translation table(Default: 1). Assign the correct genetic code based on your organism, \
						[please refer: https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi]")
	parser.add_argument("-mA","--min-AA-length",dest="min_AA_length",default="20",required=False,
						help="The minimal length of predicted peptides,default 20", type=int)
	parser.add_argument("--dependence_test", dest="dependence_test", required=False, type=str, default="none",
						choices=["none", "mic", "pcc"],
						help="the method for measuring the dependence between frame1 and frame2. \n \
							This test could help determine whether the combined p-values should be ajusted to account for the dependence between two test (i.e. F0 vs F1 and F0 vs F2). \n \
							mic: Maximal Information Coefficient; pcc: Pearson Correlation Coefficient.")
	parser.add_argument("--stouffer_adj", dest="stouffer_adj", required=False, type=str, default="none",
						choices=["none","nyholt", "liji", "gao", "galwey"],
						help="the method for adjustment the cominbed p-values to account for the dependence between two tests (i.e. F0 vs F1 and F0 vs F2). \n \
							see details at: https://search.r-project.org/CRAN/refmans/poolr/html/stouffer.html")
	parser.add_argument("--pval_adj", dest="pval_adj", required=False, type=str, default="fdr_bh",
						choices=["fdr_bh","bonferroni","sidak","holm-sidak","holm","simes-hochberg","hommel","fdr_by","fdr_tsbh","fdr_tsbky"],
						help="the method used to correct p-values for multiple testing. default: Benjamini/Hochberg method. \n \
							see details at: https://www.statsmodels.org/devel/generated/statsmodels.stats.multitest.multipletests.html#statsmodels.stats.multitest.multipletests")
	# parser.add_argument("-P","--parallel_num",dest="parallel_num",default=1,required=False,
	#					 help="the number of threads to read the alignment file(s), \
	#					 the optimal value is the number of alignment files, default=1",type=int)
	parser.add_argument("-o","--output-name",dest="output_name",default="final_result",required=False,
						help="output file name, default: final_result", type=str)
	parser.add_argument("-outgtf","--output-gtf",dest="output_gtf",action='store_true',default=False,required=False,
						help="output the gtf file of predicted ORFs")
	parser.add_argument("-outbed","--output-bed",dest="output_bed",action='store_true',default=False,required=False,
						help="output the bed file of predicted ORFs")
	parser.add_argument('-V',"--version",action="version",version=__version__)
	args = parser.parse_args()
	if type(args.rpf_mapping_file) is str:
		if not os.path.exists(args.rpf_mapping_file):
			raise  ValueError("Error, the rpf mapping file not found: %s\n" % args.rpf_mapping_file)
		args.rpf_mapping_file = [args.rpf_mapping_file]
	if args.stranded == "yes":
		args.stranded = True
	elif args.stranded == "reverse":
		args.stranded = False
	else:
		args.stranded = None
	if args.minLength > args.maxLength:
		raise ValueError("minimum length must be <= maximum length (currently %d and %d, respectively)" % (args.minLength, args.maxLength))
	if args.minLength <= 0 or  args.maxLength <=0:
		raise ValueError("minimum length or maximum length must be larger than 0.")
	args.frame0_percent = float(args.frame0_percent)

	return args

In [16]:
from Bio.Seq import translate
from sys import stderr
def translation(seq,table=1,cds=True):
	"""
	translate the DNA to protein sequence using the translation table
	table = 1, is the standard table, ref: https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi
	"""
	if len(seq) % 3 != 0:
		stderr.write("Warning: sequence is not divisible by 3")
		seq = seq[:-(len(seq) % 3)]
	return translate(seq,table,cds=cds)

In [17]:
import re

def searchCodon(seq,START_CODON,ALTERNATIVE_START_CODON_LIST,STOP_CODON_LIST):
	"""
	search all start codon and stop codons for each frame
	:param seq:
	:return: start_codon_dict, stop_codon_dict, the key is frame.
	"""

	seq = seq.upper()

	start_regx = re.compile('|'.join(START_CODON))
	stop_regx = re.compile('|'.join(STOP_CODON_LIST))

	start_idx = [m.start(0) for m in re.finditer(start_regx,seq)]
	stop_idx = [m.start(0) for m in re.finditer(stop_regx,seq)]
	if ALTERNATIVE_START_CODON_LIST:
		alt_start_regx = re.compile('|'.join(ALTERNATIVE_START_CODON_LIST))
		alt_start_idx = [m.start(0) for m in re.finditer(alt_start_regx,seq)]
	else:
		alt_start_idx = False
	return start_idx,stop_idx,alt_start_idx

def orf_find(start_idx,stop_idx,alt_start_idx,MIN_AA_LENGTH):
	"""
	extract ORFs
	"""
	commonstop_dict = {}
	for f in (0,1,2):
		inframe_stops = [x for x in stop_idx if x%3==f]
		inframe_starts = [x for x in start_idx if x%3==f]
		if alt_start_idx:
			inframe_alt_starts = [x for x in alt_start_idx if x%3==f]
		else:
			inframe_alt_starts = None

		last_stop = -1
		for j in inframe_stops:
			tmp_starts = []
			alt_flag = 1
			if inframe_starts:
				for i in inframe_starts:
					if i < j and i > last_stop and (j-i) >= MIN_AA_LENGTH*3:
						alt_flag = 0
						tmp_starts.append(i)
			if alt_flag and inframe_alt_starts:
				for i in inframe_alt_starts:
					if i < j and i > last_stop and (j-i) >= MIN_AA_LENGTH*3:
						tmp_starts.append(i)
			last_stop = j
			if tmp_starts:
				commonstop_dict[j] = tmp_starts
	return commonstop_dict

def orf_finder(seq,START_CODON,ALTERNATIVE_START_CODON_LIST,STOP_CODON_LIST,MIN_AA_LENGTH):
	start_idx,stop_idx,alt_start_idx = searchCodon(seq,START_CODON,ALTERNATIVE_START_CODON_LIST,STOP_CODON_LIST)
	orf_dict = orf_find(start_idx,stop_idx,alt_start_idx,MIN_AA_LENGTH)
	return orf_dict

In [18]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import h5py
import os

def generate_colors(length,shiftn=0):
	"""
	generate r,b,g colors
	"""
	colors0 = ["red","blue","green"]
	colors1 = ["green","red","blue"]
	colors2 = ["blue","green","red"]
	rep_times = int(np.ceil(length/3))
	if shiftn == 0:
		colors = np.tile(colors0,rep_times)
	elif shiftn==1:
		colors = np.tile(colors1,rep_times)
	else:
		colors = np.tile(colors2,rep_times)
	return colors[:length]

def plot_ORF(ax,tpsites,orf_start,orf_stop,transcript_id,StartCodon):
	colors = generate_colors(tpsites.size,orf_start%3)
	ax.vlines(np.arange(tpsites.size),ymin=0,ymax=tpsites,colors=colors,linewidth=0.3)
	ax.tick_params(axis='x',which="both",top=False,direction='out',labelsize=15)
	ax.tick_params(axis='y',which="both",labelsize=15)
	ax.set_ylabel("P-site reads density",fontsize=18)
	ax.set_xlim((0,tpsites.size))
	ax.set_title(transcript_id+":"+str(orf_start)+"-"+str(orf_stop)+":"+StartCodon)

def plot_predicted(tpsites,orf_start,orf_stop,transcript_id,StartCodon,outname):
	fig=plt.figure(figsize=(8,4))
	## ax1
	ax1=fig.add_subplot(111)
	tpsites_orf=tpsites[int(orf_start)-1:orf_stop]
	colors = generate_colors(tpsites.size,orf_start%3)
	ax1.vlines(np.arange(tpsites_orf.size),ymin=0,ymax=tpsites_orf,colors=colors[int(orf_start)-1:orf_stop],linewidth=0.3)
	ax1.tick_params(axis='x',which="both",top=False,direction='out',labelsize=15)
	ax1.tick_params(axis='y',which="both",labelsize=15)
	ax1.set_ylabel("P-site reads density",fontsize=18)
	ax1.set_xlim((0,tpsites_orf.size))
	ax1.set_title(transcript_id+":"+str(orf_start)+"-"+str(orf_stop)+":"+StartCodon)
	plt.savefig(outname + ".pdf")


def plot_annotation(ax,tlength,start,stop,label,color):
	width = 0.15
	ax.set_xlim((0,tlength))
	ax.fill((start-1,stop,stop,start-1),
			(1+width/2,1+width/2,1-width/2,1-width/2),
			color=color,lw=0.5,zorder=20)
	ax.axhline(1,color="gray",lw=0.5)
	ax.set_frame_on(False)
	ax.xaxis.set_ticks_position("none")
	ax.yaxis.set_ticks_position("none")
	ax.set_xticks([])
	ax.set_yticks([1])
	ax.set_yticklabels([label],fontsize=18)
	ax.set_ylim((1-width/2,1+width/2))

def read_psites_array(filename,transcript_id):
	with h5py.File(filename,"r") as fin:
		k=np.array([i.decode("utf-8") if type(i)==bytes else i for i in fin["transcript_ids"][:]])
		idx = np.where(k == transcript_id)[0][0]
		v = fin["p_sites"][idx]
	return v

def plot_main(cds_start,cds_end,psites_array,orf_tstart,orf_tstop,transcript_id,StartCodon,outname):
	"""
	the main plot function
	"""
	plt.figure(figsize=(8,4))
	if cds_start is not None:
		gs = gridspec.GridSpec(3,1,height_ratios=[10,1,1],hspace=0.6,left=0.2,right=0.95)
	else:
		gs = gridspec.GridSpec(2,1,height_ratios=[11,1],hspace=0.6,left=0.2,right=0.95)

	ax1 = plt.subplot(gs[0])
	ax2 = plt.subplot(gs[1])
	plot_ORF(ax1,psites_array,orf_tstart,orf_tstop,transcript_id,StartCodon)
	plot_annotation(ax2,psites_array.size,orf_tstart,orf_tstop,"Predicted","#3994FF")

	if cds_start is not None:
		ax3 = plt.subplot(gs[2])
		plot_annotation(ax3,psites_array.size,cds_start,cds_end,"Annotated","#006DD5")
	# plt.tight_layout()
	plt.savefig(outname + ".pdf")

def main():
	from .parsing_opts import parsing_plot_orf_density
	from .loadconfig import LoadConfig
	args = parsing_plot_orf_density()
	transcripts_cds_file = os.path.join(args.annot_dir,"transcripts_cds.txt")
	cds_start = None
	cds_end = None
	with open(transcripts_cds_file) as fin:
		for line in fin:
			if line.startswith(args.transcript_id):
				_,s,e = line.strip().split("\t")
				cds_start = int(s)
				cds_end = int(e)
				break
	configIn = LoadConfig(args.config_file)
	samples = [i["samplename"] for i in configIn.configList]
	for i,s in enumerate(samples):
		if i == 0:
			psites_array = read_psites_array(s + "_psites.hd5",args.transcript_id)
		else:
			psites_array += read_psites_array(s + "_psites.hd5",args.transcript_id)
	if not args.outname:
		args.outname = "%s_%i_%i" % (args.transcript_id,args.orf_tstart,args.orf_tstop)
	if args.PlotAnnotatedORF.upper()=="YES":
		plot_main(cds_start,cds_end,psites_array,args.orf_tstart,args.orf_tstop,args.transcript_id,args.StartCodon,args.outname)
	elif args.PlotAnnotatedORF.upper() == 'NO':
		plot_predicted(psites_array,args.orf_tstart,args.orf_tstop,args.transcript_id,args.StartCodon,args.outname)
	else:
		raise IOError("Please reset your --plot-annotated-orf parameter: yes or no. default=yes")

In [35]:
from pyfasta import DuplicateHeaderException
?DuplicateHeaderException

Init signature: DuplicateHeaderException(header)
Docstring:      Common base class for all non-exit exceptions.
File:           ~/.local/lib/python3.8/site-packages/pyfasta/fasta.py
Type:           type
Subclasses:     


In [7]:
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function
from builtins import range,object
# -*- coding:UTF-8 -*-
"""
preparing the transcripts annotation files.
"""
import sys
if sys.version_info.major ==2:
	import cPickle as pickle
else:
	import pickle
	from sys import intern
import os
from pyfasta import Fasta
from time import strftime
import re

class ParsingError(Exception):
	pass

class Interval(object):
	""" basic interval """
	def __init__(self,start,end,strand):
		if end < start:
			raise ValueError('Error: end smaller than start !')

		self.start = start
		self.end = end
		self.strand = strand
		if strand == "+":
			self.start_d = start
			self.end_d = end
		elif strand == "-":
			self.start_d = end - 1
			self.end_d = start - 1
		else:
			raise ValueError('Error: strand is neither "+" nor "-" !')
		self.length = end - start

	def contain(self,start,end):
		if end < start:
			raise ValueError('Error: end smaller than start !')
		if self.start <= start and self.end >= end:
			return True
		else:
			return False

	def contain_iv(self,iv):
		return self.contain(iv.start,iv.end)

	def is_overlapped_with(self, iv):
		if self.start <= iv.start <= self.end or self.start <= iv.end <= self.end:
			return True
		else:
			return False

	def __str__(self):
		return "iv:%i-%i" % (self.start,self.end)

	__repr__ = __str__

def Interval_from_directional(start_d,end_d,strand="-"):
	if strand == "-":
		return Interval(end_d+1,start_d+1,"-")
	else:
		return Interval(start_d,end_d,"+")

def parsing_attr(attr_string):
	attr_dict = {}
	attr_split = re.findall(r'.*? ".*?";', attr_string)
	for i in attr_split:
		if i:
			k,v = i.strip().split(" ",1)
			k = intern(k)
			if v.endswith(";"): v = v[:-1]
			v = intern(v.strip('"'))
			if k == "tag":
				attr_dict.setdefault(k,[]).append(v)
			else:
				attr_dict[k] = v
	return attr_dict

def parsing_line(line):
	"""
        # GTF columns:
        # 1) chrom: str ("1", "X", "chrX", etc...)
        # 2) source : str (not used)
        # 3) feature : str ("gene", "transcript", &c)
        # 4) start : int
        # 5) end : int
        # 6) score : float or "." (not used)
        # 7) strand : "+", "-", or "."
        # 8) frame : 0, 1, 2 or "." (not used)
        # 9) attribute : key-value pairs separated by semicolons
	"""
	chrom,source,feature,start,end,score,strand,frame,attr = line.strip().split("\t",8)
	#convert to 0-based.
	iv = Interval(int(start) - 1, int(end), strand)
	field_dict = {"chrom": intern(chrom),"source":source,"feature": intern(feature),"iv":iv,"attr":parsing_attr(attr)}
	return field_dict

class Gene(object):
	""" Gene object """

	def __init__(self,field_dict):
		self.chrom = field_dict["chrom"]
		self.genomic_iv = field_dict["iv"]
		self.attr = field_dict["attr"]
		self.gene_id = self.attr["gene_id"]
		self.gene_name = self.attr.get("gene_name",self.attr.get("gene_id",None))
		self.gene_type = self.attr.get("gene_type",self.attr.get("gene_biotype",None))
		self.transcripts = []
		self.principal_transcripts = []

	def __str__(self):
		return "GeneID:%s;GeneName:%s;GeneType:%s" % (self.gene_id,self.gene_name,self.gene_type)

	__repr__ = __str__

class Transcript(object):
	""" Transcript object """

	def __init__(self,field_dict):
		self.chrom = field_dict["chrom"]
		self.genomic_iv = field_dict["iv"]
		self.attr = field_dict["attr"]
		self.gene_id = self.attr["gene_id"]
		self.gene_name = self.attr.get("gene_name",self.attr.get("gene_id",None))
		self.transcript_type = self.attr.get("transcript_type",self.attr.get("transcript_biotype",None))
		self.transcript_id = self.attr["transcript_id"]
		self.genomic_exons = []
		self.genomic_cds = []
		self.genomic_startcodon = []
		self.genomic_stopcodon = []
		self.length = 0

		self.cds = None
		self.startcodon = None
		self.stopcodon = None

	def add_feature(self,field_dict):
		feature = field_dict["feature"]
		iv = field_dict["iv"]
		if feature == "exon":
			self.genomic_exons.append(iv)
			self.length += iv.length
		elif feature == "CDS":
			self.genomic_cds.append(iv)
		elif feature == "start_codon":
			self.genomic_startcodon.append(iv)
		elif feature == "stop_codon":
			self.genomic_stopcodon.append(iv)
		else:
			raise ValueError("Error: the feature is not recognized: %s" % feature)

	def __str__(self):
		return "TranscriptID:%s;TranscriptType:%s" % (self.transcript_id,self.transcript_type)

	__repr__ = __str__

def readGTF(filename):
	if not os.path.exists(filename):
		raise IOError("\tGTF file does not exist: %s" % filename)
	sys.stdout.write("\tReading the GTF file: %s .......\n" % filename)
	gene_dict = {}
	transcript_dict = {}
	with open(filename) as fin:
		for i,line in enumerate(fin):
			if line[0] == "#" or (not line.strip()):
				continue
			field_dict = parsing_line(line)
			if field_dict["feature"] == "gene":
				gobj = Gene(field_dict)
				gene_dict[gobj.gene_id] = gobj
			elif field_dict["feature"] == "transcript":
				tobj = Transcript(field_dict)
				transcript_dict[tobj.transcript_id] = tobj
			elif field_dict["feature"] in ["exon","CDS","start_codon","stop_codon"]:
				tid = field_dict["attr"]["transcript_id"]
				try:
					transcript_dict[tid].add_feature(field_dict)
				except KeyError:
					raise ParsingError("Error in line %i. The annotation in GTF file should be three-level hierarchy of \
					                    gene => transcript => exon (or CDS)" % i)
			else:
				pass
	return gene_dict,transcript_dict

def get_chrom(name):
	if " " in name:
		return name.split()[0]
	elif "|" in name:
		return name.split("|")
	else:
		return name

class GenomeSeq(object):
	""" genomic sequence"""

	def __init__(self,filename):
		self.filename = filename
		self.fh = Fasta(filename, key_fn = get_chrom)
	def get_seq(self,chrom,start=0,end=False,strand="+"):
		if end is False:
			end = len(self.fh[chrom])
		return self.fh.sequence({"chr":chrom, "start":start, "stop": end, "strand":strand}, one_based=False)


def genomic_iv_transform(genomic_exons_sorted, genomic_ivs_sorted):
	"""
	transform the genomic interval (of cds, start and stop codons) to inner coordinate in transcript,
	transcript_exons must be sorted
	return 0-based iv
	"""

	length = 0
	innerIV = []
	for i in genomic_exons_sorted:
		for j in genomic_ivs_sorted:
			if i.contain_iv(j):
				start = length + abs(j.start_d - i.start_d)
				end = start + (j.end - j.start)
				if innerIV and start == innerIV[-1].end:
					innerIV[-1] = Interval(innerIV[-1].start, end, "+")
				else:
					innerIV.append(Interval(start,end,"+"))
		length += i.end - i.start
	if len(innerIV) == 0:
		raise ValueError("\tCan't transform the genomic interval, please check!")
	return innerIV

def transcript_pos_transform(tobj, transcript_pos):
	"""
	transform the transcript position to genomic position
	0-based
	if strand is "-", return position of strand.
	"""
	length = 0
	strand = tobj.genomic_iv.strand
	genomic_pos = -1
	for i in tobj.genomic_exons:
		if transcript_pos < length + i.length:
			if strand == "+":
				genomic_pos = transcript_pos - length + i.start
			else:
				genomic_pos = i.start_d - (transcript_pos - length)
			break
		length += i.length
	return genomic_pos

def transcript_iv_transform(tobj, transcript_iv):
	"""
	transform the transcript interval to genomic interval, zero-based.
	"""
	strand = tobj.genomic_iv.strand
	genomic_start = transcript_pos_transform(tobj,transcript_iv.start)
	genomic_end = transcript_pos_transform(tobj,transcript_iv.end-1)
	if strand == "+":
		genomic_end += 1
	else:
		genomic_end -= 1
	exons_bound = [genomic_start]


	for i in tobj.genomic_exons:
		if strand == "+":
			if genomic_start < i.start < genomic_end:
				exons_bound.append(i.start)
			if genomic_start < i.end < genomic_end:
				exons_bound.append(i.end)
		else:
			if genomic_end < i.start_d < genomic_start:
				exons_bound.append(i.start_d)
			if genomic_end < i.end_d < genomic_start:
				exons_bound.append(i.end_d)

	exons_bound.append(genomic_end)
	if len(exons_bound) % 2 != 0:
		print("error when transform the transcript interval to genomic!")
	exons_ivs = []
	for i in range(0,len(exons_bound),2):
		exons_ivs.append(Interval_from_directional(exons_bound[i],exons_bound[i+1],strand))

	return exons_ivs

def load_transcripts_pickle(pickle_file):
	if os.path.exists(pickle_file):
		sys.stdout.write("\tLoading transcripts.pickle ...\n")
		with open(pickle_file,"rb") as fin:
			gene_dict, transcript_dict = pickle.load(fin)
	else:
		raise IOError("\tError, %s file not found\n" % pickle_file)

	return gene_dict,transcript_dict

def processTranscripts(genomeFasta,gtfFile,out_dir):
	"""
	transform the genomic position into the transcript position
	"""
	pickle_file = os.path.join(out_dir,"transcripts.pickle")
	if os.path.exists(pickle_file):
		gene_dict, transcript_dict = load_transcripts_pickle(pickle_file)
		return gene_dict, transcript_dict
	else:
		gene_dict,transcript_dict = readGTF(gtfFile)
		if not os.path.exists(genomeFasta):
			raise IOError("\tError, the genomic fasta file not found: %s" % genomeFasta)
		genomic_seq = GenomeSeq(genomeFasta)
		transcript_seq_file = open(os.path.join(out_dir,"transcripts_sequence.fa"),"w") #title: transcriptid length
		transcript_cds_file = open(os.path.join(out_dir,"transcripts_cds.txt"),"w") # tid, cds_start, cds_end

		sys.stderr.write("\tProcess the transcripts ....\n")
		for tobj in transcript_dict.values():
			# store the transcript id in gene object
			if tobj.transcript_id not in gene_dict[tobj.gene_id].transcripts:
				gene_dict[tobj.gene_id].transcripts.append(tobj.transcript_id)
				if "appris_principal_1" in tobj.attr.get("tag",[]) or "appris_principal" in tobj.attr.get("tag", []):
					gene_dict[tobj.gene_id].principal_transcripts.append(tobj.transcript_id)
			# sort the exons,cds,startcodon,stopcodon according their position on genomic
			if tobj.genomic_iv.strand == "+":
				sorted_reverse = False
			else:
				sorted_reverse = True

			tobj.genomic_exons.sort(key=lambda x: x.start, reverse=sorted_reverse)
			# convert the CDS genomic coordination into transcript coordination
			if tobj.genomic_cds:
				tobj.genomic_cds.sort(key=lambda x: x.start, reverse=sorted_reverse)
				tobj.cds = genomic_iv_transform(tobj.genomic_exons, tobj.genomic_cds)
				if len(tobj.cds) > 1:
					sys.stderr.write("Warning: the CDS is discontinuous," +
					                  "only first region is used, %s\n" % tobj.transcript_id)
				tobj.cds = tobj.cds[0]
				transcript_cds_file.write("%s\t%i\t%i\n" % (tobj.transcript_id,tobj.cds.start +1,tobj.cds.end + 3))

			# start and stop codon
			if tobj.genomic_startcodon:
				tobj.genomic_startcodon.sort(key=lambda x: x.start, reverse=sorted_reverse)
				tobj.startcodon = genomic_iv_transform(tobj.genomic_exons,tobj.genomic_startcodon)
				if len(tobj.startcodon) >1:
					sys.stderr.write("Warning: the start codon is discontinuous," +
					                "only first region is used, %s\n" % tobj.transcript_id)
				tobj.startcodon = tobj.startcodon[0]
			elif tobj.cds:
				tobj.startcodon = Interval(tobj.cds.start, tobj.cds.start + 3, "+")
			else:
				pass

			if tobj.genomic_stopcodon:
				tobj.genomic_stopcodon.sort(key=lambda x: x.start, reverse=sorted_reverse)
				tobj.stopcodon = genomic_iv_transform(tobj.genomic_exons,tobj.genomic_stopcodon)
				if len(tobj.stopcodon) >1:
					sys.stderr.write("Warning: the stop codon is discontinuous," +
					                "only first region is used, %s\n" % tobj.transcript_id)
				tobj.stopcodon = tobj.stopcodon[0]
			elif tobj.cds:
				tobj.stopcodon = Interval(tobj.cds.end, tobj.cds.end + 3, "+")
			else:
				pass

			transcript_seq = [genomic_seq.get_seq(tobj.chrom,exon_iv.start,exon_iv.end,exon_iv.strand) for exon_iv in tobj.genomic_exons]
			transcript_seq = "".join(transcript_seq)
			# write the transcript sequence to file
			transcript_seq_file.write(">%s %i\n%s\n" % (tobj.transcript_id,tobj.length,transcript_seq))

		transcript_seq_file.close()
		transcript_cds_file.close()

		## dump the pickle
		sys.stderr.write("\tSaving the transcripts.pickle\n")
		with open(pickle_file,"wb") as fout:
			pickle.dump([gene_dict,transcript_dict],fout,protocol=pickle.HIGHEST_PROTOCOL)

		return gene_dict,transcript_dict

def verboseprint(printstring):
	sys.stderr.write('[%s] %s\n' % (strftime("%Y-%m-%d %H:%M:%S"), printstring))
	sys.stderr.flush()


def main():
	from .parsing_opts import parsing_transcript
	args = parsing_transcript()
	verboseprint("Preparing annotation files ...")
	processTranscripts(args.genomeFasta,args.gtfFile,args.out_dir)
	verboseprint("The step of preparing transcript annotation has been completed.")

In [9]:
import sys
if sys.version_info.major ==2:
	import cPickle as pickle
else:
	import pickle
	from sys import intern
import os
from pyfasta import Fasta
from time import strftime
import re

class ParsingError(Exception):
	pass

class Interval(object):
	""" basic interval """
	def __init__(self,start,end,strand):
		if end < start:
			raise ValueError('Error: end smaller than start !')

		self.start = start
		self.end = end
		self.strand = strand
		if strand == "+":
			self.start_d = start
			self.end_d = end
		elif strand == "-":
			self.start_d = end - 1
			self.end_d = start - 1
		else:
			raise ValueError('Error: strand is neither "+" nor "-" !')
		self.length = end - start

	def contain(self,start,end):
		if end < start:
			raise ValueError('Error: end smaller than start !')
		if self.start <= start and self.end >= end:
			return True
		else:
			return False

	def contain_iv(self,iv):
		return self.contain(iv.start,iv.end)

	def is_overlapped_with(self, iv):
		if self.start <= iv.start <= self.end or self.start <= iv.end <= self.end:
			return True
		else:
			return False

	def __str__(self):
		return "iv:%i-%i" % (self.start,self.end)

	__repr__ = __str__

def Interval_from_directional(start_d,end_d,strand="-"):
	if strand == "-":
		return Interval(end_d+1,start_d+1,"-")
	else:
		return Interval(start_d,end_d,"+")

def parsing_attr(attr_string):
	attr_dict = {}
	attr_split = re.findall(r'.*? ".*?";', attr_string)
	for i in attr_split:
		if i:
			k,v = i.strip().split(" ",1)
			k = intern(k)
			if v.endswith(";"): v = v[:-1]
			v = intern(v.strip('"'))
			if k == "tag":
				attr_dict.setdefault(k,[]).append(v)
			else:
				attr_dict[k] = v
	return attr_dict

def parsing_line(line):
	"""
        # GTF columns:
        # 1) chrom: str ("1", "X", "chrX", etc...)
        # 2) source : str (not used)
        # 3) feature : str ("gene", "transcript", &c)
        # 4) start : int
        # 5) end : int
        # 6) score : float or "." (not used)
        # 7) strand : "+", "-", or "."
        # 8) frame : 0, 1, 2 or "." (not used)
        # 9) attribute : key-value pairs separated by semicolons
	"""
	chrom,source,feature,start,end,score,strand,frame,attr = line.strip().split("\t",8)
	#convert to 0-based.
	iv = Interval(int(start) - 1, int(end), strand)
	field_dict = {"chrom": intern(chrom),"source":source,"feature": intern(feature),"iv":iv,"attr":parsing_attr(attr)}
	return field_dict

class Gene(object):
	""" Gene object """

	def __init__(self,field_dict):
		self.chrom = field_dict["chrom"]
		self.genomic_iv = field_dict["iv"]
		self.attr = field_dict["attr"]
		self.gene_id = self.attr["gene_id"]
		self.gene_name = self.attr.get("gene_name",self.attr.get("gene_id",None))
		self.gene_type = self.attr.get("gene_type",self.attr.get("gene_biotype",None))
		self.transcripts = []
		self.principal_transcripts = []

	def __str__(self):
		return "GeneID:%s;GeneName:%s;GeneType:%s" % (self.gene_id,self.gene_name,self.gene_type)

	__repr__ = __str__

class Transcript(object):
	""" Transcript object """

	def __init__(self,field_dict):
		self.chrom = field_dict["chrom"]
		self.genomic_iv = field_dict["iv"]
		self.attr = field_dict["attr"]
		self.gene_id = self.attr["gene_id"]
		self.gene_name = self.attr.get("gene_name",self.attr.get("gene_id",None))
		self.transcript_type = self.attr.get("transcript_type",self.attr.get("transcript_biotype",None))
		self.transcript_id = self.attr["transcript_id"]
		self.genomic_exons = []
		self.genomic_cds = []
		self.genomic_startcodon = []
		self.genomic_stopcodon = []
		self.length = 0

		self.cds = None
		self.startcodon = None
		self.stopcodon = None

	def add_feature(self,field_dict):
		feature = field_dict["feature"]
		iv = field_dict["iv"]
		if feature == "exon":
			self.genomic_exons.append(iv)
			self.length += iv.length
		elif feature == "CDS":
			self.genomic_cds.append(iv)
		elif feature == "start_codon":
			self.genomic_startcodon.append(iv)
		elif feature == "stop_codon":
			self.genomic_stopcodon.append(iv)
		else:
			raise ValueError("Error: the feature is not recognized: %s" % feature)

	def __str__(self):
		return "TranscriptID:%s;TranscriptType:%s" % (self.transcript_id,self.transcript_type)

	__repr__ = __str__

def readGTF(filename):
	if not os.path.exists(filename):
		raise IOError("\tGTF file does not exist: %s" % filename)
	sys.stdout.write("\tReading the GTF file: %s .......\n" % filename)
	gene_dict = {}
	transcript_dict = {}
	with open(filename) as fin:
		for i,line in enumerate(fin):
			if line[0] == "#" or (not line.strip()):
				continue
			field_dict = parsing_line(line)
			if field_dict["feature"] == "gene":
				gobj = Gene(field_dict)
				gene_dict[gobj.gene_id] = gobj
			elif field_dict["feature"] == "transcript":
				tobj = Transcript(field_dict)
				transcript_dict[tobj.transcript_id] = tobj
			elif field_dict["feature"] in ["exon","CDS","start_codon","stop_codon"]:
				tid = field_dict["attr"]["transcript_id"]
				try:
					transcript_dict[tid].add_feature(field_dict)
				except KeyError:
					raise ParsingError("Error in line %i. The annotation in GTF file should be three-level hierarchy of \
					                    gene => transcript => exon (or CDS)" % i)
			else:
				pass
	return gene_dict,transcript_dict

def get_chrom(name):
	if " " in name:
		return name.split()[0]
	elif "|" in name:
		return name.split("|")
	else:
		return name

class GenomeSeq(object):
	""" genomic sequence"""

	def __init__(self,filename):
		self.filename = filename
		self.fh = Fasta(filename, key_fn = get_chrom)
	def get_seq(self,chrom,start=0,end=False,strand="+"):
		if end is False:
			end = len(self.fh[chrom])
		return self.fh.sequence({"chr":chrom, "start":start, "stop": end, "strand":strand}, one_based=False)


def genomic_iv_transform(genomic_exons_sorted, genomic_ivs_sorted):
	"""
	transform the genomic interval (of cds, start and stop codons) to inner coordinate in transcript,
	transcript_exons must be sorted
	return 0-based iv
	"""

	length = 0
	innerIV = []
	for i in genomic_exons_sorted:
		for j in genomic_ivs_sorted:
			if i.contain_iv(j):
				start = length + abs(j.start_d - i.start_d)
				end = start + (j.end - j.start)
				if innerIV and start == innerIV[-1].end:
					innerIV[-1] = Interval(innerIV[-1].start, end, "+")
				else:
					innerIV.append(Interval(start,end,"+"))
		length += i.end - i.start
	if len(innerIV) == 0:
		raise ValueError("\tCan't transform the genomic interval, please check!")
	return innerIV

def transcript_pos_transform(tobj, transcript_pos):
	"""
	transform the transcript position to genomic position
	0-based
	if strand is "-", return position of strand.
	"""
	length = 0
	strand = tobj.genomic_iv.strand
	genomic_pos = -1
	for i in tobj.genomic_exons:
		if transcript_pos < length + i.length:
			if strand == "+":
				genomic_pos = transcript_pos - length + i.start
			else:
				genomic_pos = i.start_d - (transcript_pos - length)
			break
		length += i.length
	return genomic_pos

def transcript_iv_transform(tobj, transcript_iv):
	"""
	transform the transcript interval to genomic interval, zero-based.
	"""
	strand = tobj.genomic_iv.strand
	genomic_start = transcript_pos_transform(tobj,transcript_iv.start)
	genomic_end = transcript_pos_transform(tobj,transcript_iv.end-1)
	if strand == "+":
		genomic_end += 1
	else:
		genomic_end -= 1
	exons_bound = [genomic_start]


	for i in tobj.genomic_exons:
		if strand == "+":
			if genomic_start < i.start < genomic_end:
				exons_bound.append(i.start)
			if genomic_start < i.end < genomic_end:
				exons_bound.append(i.end)
		else:
			if genomic_end < i.start_d < genomic_start:
				exons_bound.append(i.start_d)
			if genomic_end < i.end_d < genomic_start:
				exons_bound.append(i.end_d)

	exons_bound.append(genomic_end)
	if len(exons_bound) % 2 != 0:
		print("error when transform the transcript interval to genomic!")
	exons_ivs = []
	for i in range(0,len(exons_bound),2):
		exons_ivs.append(Interval_from_directional(exons_bound[i],exons_bound[i+1],strand))

	return exons_ivs

def load_transcripts_pickle(pickle_file):
	if os.path.exists(pickle_file):
		sys.stdout.write("\tLoading transcripts.pickle ...\n")
		with open(pickle_file,"rb") as fin:
			gene_dict, transcript_dict = pickle.load(fin)
	else:
		raise IOError("\tError, %s file not found\n" % pickle_file)

	return gene_dict,transcript_dict

def processTranscripts(genomeFasta,gtfFile,out_dir):
	"""
	transform the genomic position into the transcript position
	"""
	pickle_file = os.path.join(out_dir,"transcripts.pickle")
	if os.path.exists(pickle_file):
		gene_dict, transcript_dict = load_transcripts_pickle(pickle_file)
		return gene_dict, transcript_dict
	else:
		gene_dict,transcript_dict = readGTF(gtfFile)
		if not os.path.exists(genomeFasta):
			raise IOError("\tError, the genomic fasta file not found: %s" % genomeFasta)
		genomic_seq = GenomeSeq(genomeFasta)
		transcript_seq_file = open(os.path.join(out_dir,"transcripts_sequence.fa"),"w") #title: transcriptid length
		transcript_cds_file = open(os.path.join(out_dir,"transcripts_cds.txt"),"w") # tid, cds_start, cds_end

		sys.stderr.write("\tProcess the transcripts ....\n")
		for tobj in transcript_dict.values():
			# store the transcript id in gene object
			if tobj.transcript_id not in gene_dict[tobj.gene_id].transcripts:
				gene_dict[tobj.gene_id].transcripts.append(tobj.transcript_id)
				if "appris_principal_1" in tobj.attr.get("tag",[]) or "appris_principal" in tobj.attr.get("tag", []):
					gene_dict[tobj.gene_id].principal_transcripts.append(tobj.transcript_id)
			# sort the exons,cds,startcodon,stopcodon according their position on genomic
			if tobj.genomic_iv.strand == "+":
				sorted_reverse = False
			else:
				sorted_reverse = True

			tobj.genomic_exons.sort(key=lambda x: x.start, reverse=sorted_reverse)
			# convert the CDS genomic coordination into transcript coordination
			if tobj.genomic_cds:
				tobj.genomic_cds.sort(key=lambda x: x.start, reverse=sorted_reverse)
				tobj.cds = genomic_iv_transform(tobj.genomic_exons, tobj.genomic_cds)
				if len(tobj.cds) > 1:
					sys.stderr.write("Warning: the CDS is discontinuous," +
					                  "only first region is used, %s\n" % tobj.transcript_id)
				tobj.cds = tobj.cds[0]
				transcript_cds_file.write("%s\t%i\t%i\n" % (tobj.transcript_id,tobj.cds.start +1,tobj.cds.end + 3))

			# start and stop codon
			if tobj.genomic_startcodon:
				tobj.genomic_startcodon.sort(key=lambda x: x.start, reverse=sorted_reverse)
				tobj.startcodon = genomic_iv_transform(tobj.genomic_exons,tobj.genomic_startcodon)
				if len(tobj.startcodon) >1:
					sys.stderr.write("Warning: the start codon is discontinuous," +
					                "only first region is used, %s\n" % tobj.transcript_id)
				tobj.startcodon = tobj.startcodon[0]
			elif tobj.cds:
				tobj.startcodon = Interval(tobj.cds.start, tobj.cds.start + 3, "+")
			else:
				pass

			if tobj.genomic_stopcodon:
				tobj.genomic_stopcodon.sort(key=lambda x: x.start, reverse=sorted_reverse)
				tobj.stopcodon = genomic_iv_transform(tobj.genomic_exons,tobj.genomic_stopcodon)
				if len(tobj.stopcodon) >1:
					sys.stderr.write("Warning: the stop codon is discontinuous," +
					                "only first region is used, %s\n" % tobj.transcript_id)
				tobj.stopcodon = tobj.stopcodon[0]
			elif tobj.cds:
				tobj.stopcodon = Interval(tobj.cds.end, tobj.cds.end + 3, "+")
			else:
				pass

			transcript_seq = [genomic_seq.get_seq(tobj.chrom,exon_iv.start,exon_iv.end,exon_iv.strand) for exon_iv in tobj.genomic_exons]
			transcript_seq = "".join(transcript_seq)
			# write the transcript sequence to file
			transcript_seq_file.write(">%s %i\n%s\n" % (tobj.transcript_id,tobj.length,transcript_seq))

		transcript_seq_file.close()
		transcript_cds_file.close()

		## dump the pickle
		sys.stderr.write("\tSaving the transcripts.pickle\n")
		with open(pickle_file,"wb") as fout:
			pickle.dump([gene_dict,transcript_dict],fout,protocol=pickle.HIGHEST_PROTOCOL)

		return gene_dict,transcript_dict

def verboseprint(printstring):
	sys.stderr.write('[%s] %s\n' % (strftime("%Y-%m-%d %H:%M:%S"), printstring))
	sys.stderr.flush()


def main():
	from .parsing_opts import parsing_transcript
	args = parsing_transcript()
	verboseprint("Preparing annotation files ...")
	processTranscripts(args.genomeFasta,args.gtfFile,args.out_dir)
	verboseprint("The step of preparing transcript annotation has been completed.")

In [10]:
processTranscripts('/research_jude/rgs01_jude/groups/blancgrp/projects/rRNA_variation/common/metagene_analysis/Mus_musculus.GRCm39.dna.primary_assembly.rdna.fa', 
                   '/research_jude/rgs01_jude/groups/blancgrp/projects/rRNA_variation/common/metagene_analysis/Mus_musculus.GRCm39.104.rdna_rn18s.gtf',
                   '/research_jude/rgs01_jude/groups/blancgrp/projects/rRNA_variation/common/metagene_analysis/output')

	Loading transcripts.pickle ...


({'ENSMUSG00000102628': GeneID:ENSMUSG00000102628;GeneName:Gm37671;GeneType:TEC,
  'ENSMUSG00000100595': GeneID:ENSMUSG00000100595;GeneName:Gm19087;GeneType:processed_pseudogene,
  'ENSMUSG00000097426': GeneID:ENSMUSG00000097426;GeneName:Gm8941;GeneType:processed_pseudogene,
  'ENSMUSG00000104478': GeneID:ENSMUSG00000104478;GeneName:Gm38212;GeneType:TEC,
  'ENSMUSG00000104385': GeneID:ENSMUSG00000104385;GeneName:Gm7449;GeneType:processed_pseudogene,
  'ENSMUSG00000086053': GeneID:ENSMUSG00000086053;GeneName:Gm15178;GeneType:lncRNA,
  'ENSMUSG00000101231': GeneID:ENSMUSG00000101231;GeneName:Gm28283;GeneType:processed_pseudogene,
  'ENSMUSG00000102135': GeneID:ENSMUSG00000102135;GeneName:Gm37108;GeneType:processed_pseudogene,
  'ENSMUSG00000103282': GeneID:ENSMUSG00000103282;GeneName:Gm37275;GeneType:processed_pseudogene,
  'ENSMUSG00000101097': GeneID:ENSMUSG00000101097;GeneName:Gm6679;GeneType:processed_pseudogene,
  'ENSMUSG00000100764': GeneID:ENSMUSG00000100764;GeneName:Gm29155;Gene

In [13]:
from collections import Counter,defaultdict
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pysam
from matplotlib.backends.backend_pdf import PdfPages
#from prepare_transcripts import *
#from .detectORF import extract_frame,test_frame
#from .parsing_opts import parsing_metaplots
import os
import sys

In [39]:
import pyfasta
import RiboMiner

In [42]:
import pandas as pd
import numpy as np

print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

re==2.2.1
argparse==1.1
matplotlib==3.3.4
numpy==1.22.3
pysam==0.16.0.1
h5py==2.10.0
pandas==1.2.4


Traceback (most recent call last):
  File "/home/nmishra/.conda/envs/RiboMiner/bin/metaplots", line 33, in <module>
    sys.exit(load_entry_point('RiboCode==1.2.15', 'console_scripts', 'metaplots')())
  File "/home/nmishra/.conda/envs/RiboMiner/bin/metaplots", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/home/nmishra/.conda/envs/RiboMiner/lib/python3.7/site-packages/importlib_metadata/__init__.py", line 209, in load
    module = import_module(match.group('module'))
  File "/home/nmishra/.conda/envs/RiboMiner/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1006, in _gcd_import
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_extern